# Building Predictive Models - Part 1

In [1]:
import pandas as pd
import numpy as np
import os

### Import Data

In [2]:
# set the path of processed data
processed_data_path = os.path.join(os.path.pardir, 'data', 'processed')
train_file_path = os.path.join(processed_data_path, 'train.csv')
test_file_path = os.path.join(processed_data_path, 'test.csv')

In [3]:
train_df = pd.read_csv(train_file_path, index_col = 'PassengerId')
test_df = pd.read_csv(test_file_path, index_col = 'PassengerId')

In [4]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 891 entries, 1 to 891
Data columns (total 21 columns):
Survived         891 non-null int64
Pclass           891 non-null int64
AgeBand          891 non-null int64
FareBand         891 non-null int64
IsSenior         891 non-null int64
FamilySize       891 non-null int64
IsAlone          891 non-null int64
IsMother         891 non-null int64
IsMale           891 non-null int64
IsAdult          891 non-null int64
DeckLevel        891 non-null int64
Title_Lady       891 non-null int64
Title_Master     891 non-null int64
Title_Miss       891 non-null int64
Title_Mr         891 non-null int64
Title_Mrs        891 non-null int64
Title_Officer    891 non-null int64
Title_Sir        891 non-null int64
Embarked_C       891 non-null int64
Embarked_Q       891 non-null int64
Embarked_S       891 non-null int64
dtypes: int64(21)
memory usage: 153.1 KB


In [5]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 418 entries, 892 to 1309
Data columns (total 20 columns):
Pclass           418 non-null int64
AgeBand          418 non-null int64
FareBand         418 non-null int64
IsSenior         418 non-null int64
FamilySize       418 non-null int64
IsAlone          418 non-null int64
IsMother         418 non-null int64
IsMale           418 non-null int64
IsAdult          418 non-null int64
DeckLevel        418 non-null int64
Title_Lady       418 non-null int64
Title_Master     418 non-null int64
Title_Miss       418 non-null int64
Title_Mr         418 non-null int64
Title_Mrs        418 non-null int64
Title_Officer    418 non-null int64
Title_Sir        418 non-null int64
Embarked_C       418 non-null int64
Embarked_Q       418 non-null int64
Embarked_S       418 non-null int64
dtypes: int64(20)
memory usage: 68.6 KB


### Data Preparation

In [51]:
X = train_df.loc[:, 'Pclass':].values.astype('float')
y = train_df['Survived'].ravel()
print(X.shape, y.shape)

(891, 20) (891,)


In [52]:
# train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(712, 20) (712,)
(179, 20) (179,)


In [53]:
# average survival in train and test
print('mean survival in train: {0:0.3f}'.format(np.mean(y_train)))
print('mean survival in test: {0:0.3f}'.format(np.mean(y_test)))

mean survival in train: 0.383
mean survival in test: 0.385


#### Check Scikit-Learn Version

In [9]:
import sklearn
sklearn.__version__

'0.20.0'

Make sure you have Scikit-Learn v0.19 or newer. Otherwise update it and restart kernel.

In [10]:
#!conda update -y scikit-learn

### Baseline Model

In [11]:
# import function
from sklearn.dummy import DummyClassifier

In [12]:
# create baseline model
model_dummy = DummyClassifier(strategy = 'most_frequent', random_state = 0)

In [13]:
# train model
model_dummy.fit(X_train, y_train)

DummyClassifier(constant=None, random_state=0, strategy='most_frequent')

In [14]:
# default score - accuracy
print('score for baseline model: {0:0.2f}'.format(model_dummy.score(X_test, y_test)))

score for baseline model: 0.61


In [15]:
# performance metrics
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

In [16]:
# accuracy score
print('accuracy for baseline model: {0:0.2f}'.format(accuracy_score(y_test, model_dummy.predict(X_test))))

accuracy for baseline model: 0.61


In [17]:
# confusion matrix
print('confusion matrix for baseline model: \n {0}'.format(confusion_matrix(y_test, model_dummy.predict(X_test))))

confusion matrix for baseline model: 
 [[110   0]
 [ 69   0]]


In [18]:
# precision and recall scores
print('precision for baseline model: {0:0.2f}'.format(precision_score(y_test, model_dummy.predict(X_test))))
print('recall for baseline model: {0:0.2f}'.format(recall_score(y_test, model_dummy.predict(X_test))))

precision for baseline model: 0.00
recall for baseline model: 0.00


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


### First Kaggle Submission

In [54]:
# converting to matrix
test_X = test_df.values.astype('float')

In [20]:
# get predictions
predictions = model_dummy.predict(test_X)

In [21]:
df_submission = pd.DataFrame({ 'PassengerId': test_df.index, 'Survived': predictions })
df_submission.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,0


In [22]:
submission_data_path = os.path.join(os.path.pardir, 'data', 'external')
submission_file_path = os.path.join(submission_data_path, '01_dummy.csv')
print(submission_file_path)

..\data\external\01_dummy.csv


In [23]:
df_submission.to_csv(submission_file_path, index = False)

In [24]:
def get_submission_file(model, filename, message):
    # converting to matrix
    test_X = test_df.values.astype('float')
    # make predictions
    predictions = model.predict(test_X)
    # submission dataframe
    df_submission = pd.DataFrame({ 'PassengerId': test_df.index, 'Survived': predictions })
    # submission file
    submission_data_path = os.path.join(os.path.pardir, 'data', 'external')
    submission_file_path = os.path.join(submission_data_path, filename)
    # write to file
    df_submission.to_csv(submission_file_path, index = False)
    # submit using Kaggle API
    from subprocess import call
    call(['kaggle', 'competitions', 'submit', 'titanic', '-f', submission_file_path, '-m', message])

In [46]:
# get submission file
get_submission_file(model_dummy, '01_dummy.csv', 'Baseline Model Submission')

### Logistic Regression Model

In [25]:
# import function
from sklearn.linear_model import LogisticRegression

In [26]:
# create model
model_lr_1 = LogisticRegression(random_state = 0, solver = 'liblinear')

In [27]:
# train model
model_lr_1.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [28]:
# evaluate model
print('score for logistic regression - version 1: {0:0.2f}'.format(model_lr_1.score(X_test, y_test)))

score for logistic regression - version 1: 0.85


In [29]:
# performance metrics
# accuracy
print('accuracy for logistic regression - version 1: {0:0.2f}'.format(accuracy_score(y_test, model_lr_1.predict(X_test))))
# confusion matrix
print('confusion matrix logistic regression - version 1: \n {0}'.format(confusion_matrix(y_test, model_lr_1.predict(X_test))))
# precision
print('precision for logistic regression - version 1: {0:0.2f}'.format(precision_score(y_test, model_lr_1.predict(X_test))))
# recall
print('recall for logistic regression - version 1: {0:0.2f}'.format(recall_score(y_test, model_lr_1.predict(X_test))))

accuracy for logistic regression - version 1: 0.85
confusion matrix logistic regression - version 1: 
 [[95 15]
 [12 57]]
precision for logistic regression - version 1: 0.79
recall for logistic regression - version 1: 0.83


In [30]:
# model coefficients
model_lr_1.coef_

array([[-0.56477773, -0.28735993,  0.39284498, -0.48425076, -0.52283117,
        -0.10851157,  0.18309001, -0.85124772, -0.15176182, -0.07288012,
         0.23138359,  1.8108619 ,  1.00191631, -1.0379867 ,  1.43128147,
        -0.3446081 , -0.92832066,  0.89744794,  0.76455096,  0.50252891]])

### Second Kaggle Submission

In [31]:
# get submission file
get_submission_file(model_lr_1, '02_lr.csv', 'Logistic Regression Model v1 Submission')

# Building Predictive Models - Part 2

### Hyperparameter Optimization

In [32]:
# base model
model_lr = LogisticRegression(random_state = 0, solver = 'liblinear')

In [33]:
# import function
from sklearn.model_selection import GridSearchCV

In [44]:
# create parameter grid
parameters = {
    'C': [ 0.1, 0.2, 0.5, 1.0, 10.0, 50.0, 100.0, 1000.0 ],
    'penalty': [ 'l1', 'l2' ]
}
clf = GridSearchCV(model_lr, param_grid = parameters, cv = 5) # 5-fold crossvalidation

In [45]:
clf.fit(X, y)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.1, 0.2, 0.5, 1.0, 10.0, 50.0, 100.0, 1000.0], 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [46]:
clf.best_params_

{'C': 1.0, 'penalty': 'l1'}

In [47]:
print('best score: {0:0.2f}'.format(clf.best_score_))

best score: 0.82


In [48]:
# evaluate model
print('score for logistic regression - version 2: {0:0.2f}'.format(clf.score(X_test, y_test)))

score for logistic regression - version 2: 0.85


### Third Kaggle Submission

In [39]:
# get submission file
get_submission_file(clf, '03_lr.csv', 'Logistic Regression Model v2 Submission')

### Feature Normalization and Standardization

In [49]:
# import function
from sklearn.preprocessing import MinMaxScaler, StandardScaler

#### Feature Normalization

In [55]:
# feature normalization
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [56]:
X_scaled[:,0].min(), X_scaled[:,0].max()

(0.0, 1.0)

In [57]:
# normalize test data
X_test_scaled = scaler.transform(X_test)

#### Create Model after Normalization

In [58]:
model_lr = LogisticRegression(random_state = 0, solver = 'liblinear')
parameters = {
    'C': [ 0.1, 0.2, 0.5, 1.0, 10.0, 50.0, 100.0, 1000.0 ],
    'penalty': [ 'l1', 'l2' ]
}
clf = GridSearchCV(model_lr, param_grid = parameters, cv = 5)
clf.fit(X_scaled, y)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.1, 0.2, 0.5, 1.0, 10.0, 50.0, 100.0, 1000.0], 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [59]:
clf.best_params_

{'C': 1.0, 'penalty': 'l2'}

In [60]:
clf.best_score_

0.8249158249158249

In [61]:
# evaluate model
print('score for logistic regression - after normalization: {0:0.2f}'.format(clf.score(X_test_scaled, y_test)))

score for logistic regression - after normalization: 0.85


#### Submit to Kaggle

In [62]:
# get submission file
get_submission_file(clf, '04_lr.csv', 'Logistic Regression Model (Post Normalization) Submission')

#### Feature Standardization

In [63]:
# feature standardization
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_test_scaled = scaler.transform(X_test)

#### Create Model after Standardization

In [64]:
model_lr = LogisticRegression(random_state = 0, solver = 'liblinear')
parameters = {
    'C': [ 0.1, 0.2, 0.5, 1.0, 10.0, 50.0, 100.0, 1000.0 ],
    'penalty': [ 'l1', 'l2' ]
}
clf = GridSearchCV(model_lr, param_grid = parameters, cv = 5)
clf.fit(X_scaled, y)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.1, 0.2, 0.5, 1.0, 10.0, 50.0, 100.0, 1000.0], 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [65]:
clf.best_params_

{'C': 0.2, 'penalty': 'l1'}

In [66]:
clf.best_score_

0.8294051627384961

In [67]:
# evaluate model
print('score for logistic regression - after standardization: {0:0.2f}'.format(clf.score(X_test_scaled, y_test)))

score for logistic regression - after standardization: 0.85


#### Submit to Kaggle

In [68]:
# get submission file
get_submission_file(clf, '05_lr.csv', 'Logistic Regression Model (Post Standardization) Submission')

### Model Persistance

In [69]:
# import pickle library
import pickle

In [70]:
# create file paths
model_file_path = os.path.join(os.path.pardir, 'models', 'lr_model.pkl')
print(model_file_path)
scaler_file_path = os.path.join(os.path.pardir, 'models', 'lr_scaler.pkl')
print(scaler_file_path)

..\models\lr_model.pkl
..\models\lr_scaler.pkl


In [71]:
# open files to write
model_file_pickle = open(model_file_path, 'wb')
scaler_file_pickle = open(scaler_file_path, 'wb')

# persist the model and scaler
pickle.dump(clf, model_file_pickle)
pickle.dump(scaler, scaler_file_pickle)

# close the files
model_file_pickle.close()
scaler_file_pickle.close()

#### Load Persisted Files

In [72]:
# open files in read mode
model_file_pickle = open(model_file_path, 'rb')
scaler_file_pickle = open(scaler_file_path, 'rb')

# load model and scaler
clf_loaded = pickle.load(model_file_pickle)
scaler_loaded = pickle.load(scaler_file_pickle)

# close the files
model_file_pickle.close()
scaler_file_pickle.close()

In [73]:
clf_loaded

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.1, 0.2, 0.5, 1.0, 10.0, 50.0, 100.0, 1000.0], 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [74]:
scaler_loaded

StandardScaler(copy=True, with_mean=True, with_std=True)

In [75]:
X_test_scaled = scaler_loaded.transform(X_test)
print('score for persisted logistic regression: {0:0.2f}'.format(clf_loaded.score(X_test_scaled, y_test)))

score for persisted logistic regression: 0.85
